<a href="https://colab.research.google.com/github/Fernandobdsantos/POSML/blob/main/Snowflake_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi "uvicorn[standard]" pandas snowflake-sqlalchemy pyngrok nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.5/70.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.1/453.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 5.7 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
userdata.get('SNOWFLAKE_ACCT')

'zappzjt-rcb40816'

In [ ]:
# ==============================================================================
# CÉLULA 1: INSTALAÇÃO DAS DEPENDÊNCIAS
# ==============================================================================
!pip install fastapi "uvicorn[standard]" pandas snowflake-sqlalchemy pyngrok nest_asyncio

# ==============================================================================
# CÉLULA 2: CÓDIGO DA API (MÉTODO RECOMENDADO PARA COLAB)
# ==============================================================================
import os
import uvicorn
import nest_asyncio
import pandas as pd
from pyngrok import ngrok
from sqlalchemy import create_engine, text
from typing import Optional
from fastapi import FastAPI, HTTPException, Query, status
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from google.colab import userdata # Importa a biblioteca do Colab

# --- INÍCIO DA CONFIGURAÇÃO DA API ---
app = FastAPI(
    title="BookScraper API",
    version="1.0.0",
    description="API para servir dados do Snowflake rodando no Google Colab"
)

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# --- CONEXÃO COM O SNOWFLAKE USANDO userdata.get() ---
df_books = pd.DataFrame()
books_dict = {}

try:
    # Usando userdata.get() para buscar os secrets de forma segura no Colab
    # Este bloco já falhará com um erro claro se algum secret não existir.
    engine = create_engine(
        "snowflake://{user}:{pwd}@{acct}/{db}/{schema}?warehouse={wh}&role={role}".format(
            user=userdata.get("SNOWFLAKE_USER"),
            pwd=userdata.get("SNOWFLAKE_PWD"),
            acct=userdata.get("SNOWFLAKE_ACCT"),
            db=userdata.get("SNOWFLAKE_DB"),
            schema=userdata.get("SNOWFLAKE_SCHEMA"),
            wh=userdata.get("SNOWFLAKE_WH"),
            role=userdata.get("SNOWFLAKE_ROLE")
        )
    )



    QUERY = text("SELECT id, title, category, price, rating FROM DB_SCRAPE.SC_SCRAPE.TB_BOOKS_TO_SCRAPE")
    df_books = pd.read_sql(QUERY, engine)
    df_books.set_index("id", inplace=True)
    books_dict = df_books.to_dict(orient="index")
    print("✅ Conexão com o Snowflake bem-sucedida e dados carregados.")

except Exception as e:
    # O erro agora será mais específico (ex: SecretNotFoundError, InterfaceError, etc.)
    print(f"❌ Erro crítico ao conectar ou buscar dados no Snowflake: {e}")
    print("A API continuará rodando com dados vazios.")


# --- ENDPOINTS DA API ---
# (Seus endpoints continuam aqui, sem alterações)
@app.get("/")
async def root():
    return {"message": "Bem-vindo. Anexe /docs ao final da URL do ngrok para a documentação."}

@app.get("/api/v1/health")
async def health_check():
    try:
        # Recriamos o engine para o health check para garantir que está funcional
        engine_check = create_engine("snowflake://{user}:{pwd}@{acct}/".format(user=userdata.get("SNOWFLAKE_USER"), pwd=userdata.get("SNOWFLAKE_PWD"), acct=userdata.get("SNOWFLAKE_ACCT")))
        with engine_check.connect() as conn:
            conn.execute(text("SELECT 1"))
        return JSONResponse(content={"status_code": status.HTTP_200_OK, "message": "API e Snowflake funcionais"})
    except Exception as e:
        return JSONResponse(content={"status_code": status.HTTP_503_SERVICE_UNAVAILABLE, "message": f"Erro no Snowflake: {str(e)}"})

# ... (cole o restante dos seus endpoints aqui) ...
@app.get("/api/v1/books")
async def list_books():
    if df_books.empty: return []
    return [book["title"] for book in df_books.to_dict(orient="records")]

@app.get("/api/v1/books/search")
async def search_books(
    title: Optional[str] = Query(None, description="Título parcial"),
    category: Optional[str] = Query(None, description="Categoria do livro")
):
    if df_books.empty: return []
    filtered = df_books
    if title:
        filtered = filtered[filtered["title"].str.contains(title, case=False, na=False)]
    if category:
        filtered = filtered[filtered["category"].str.contains(category, case=False, na=False)]
    return filtered.to_dict(orient="records")

@app.get("/api/v1/books/top-rated")
async def list_titles_top_rated(limit: int = 10):
    if df_books.empty: return []
    top_books = df_books.sort_values(by="rating", ascending=False).head(limit)
    return top_books["title"].tolist()

@app.get("/api/v1/books/price-range")
async def price_range(min: float, max: float):
    if df_books.empty: return []
    filtered = df_books[(df_books["price"] >= min) & (df_books["price"] <= max)]
    return filtered["title"].tolist()

@app.get("/api/v1/stats/overview")
async def collection_statistics():
    if df_books.empty: return {}
    return {
        "total_livros": len(df_books),
        "preço_medio": round(df_books["price"].mean(), 2),
        "distribuição_ratings": df_books["rating"].value_counts().sort_index().to_dict()
    }

@app.get("/api/v1/stats/categories")
async def category_statistics():
    if df_books.empty: return []
    grouped = df_books.groupby("category").agg(
        total_livros=("title", "count"),
        preco_medio=("price", "mean"),
        preco_minimo=("price", "min"),
        preco_maximo=("price", "max")
    ).round(2)
    return grouped.reset_index().to_dict(orient="records")

@app.get("/api/v1/books/{book_id}")
async def get_book(book_id: int):
    book = books_dict.get(book_id)
    if not book:
        raise HTTPException(status_code=404, detail="Livro não encontrado")
    return book

@app.get("/api/v1/categories")
async def list_categories():
    if df_books.empty: return []
    return sorted(df_books["category"].unique().tolist())

# --- INICIALIZAÇÃO DO SERVIDOR COM NGROK ---
nest_asyncio.apply()

try:
    # **NOVA LINHA**: Autentica no ngrok usando o secret
    ngrok.set_auth_token(userdata.get("NGROK_AUTHTOKEN"))

    # Inicia o túnel do ngrok na porta 8000
    public_url = ngrok.connect(8000)
    print(f"🚀 API rodando em: {public_url}")

    # Inicia o servidor uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

except Exception as e:
    print(f"❌ Erro ao iniciar o ngrok ou o servidor: {e}")

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

✅ Conexão com o Snowflake bem-sucedida e dados carregados.
🚀 API rodando em: NgrokTunnel: "https://aea9e2e73690.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [320]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     163.116.230.117:0 - "GET / HTTP/1.1" 200 OK
INFO:     163.116.230.117:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     163.116.230.117:0 - "GET /api/v1/stats/overwiew HTTP/1.1" 404 Not Found
INFO:     163.116.230.117:0 - "GET /api/v1/stats/categories HTTP/1.1" 200 OK
INFO:     163.116.230.117:0 - "GET /api/v1/stats/overview HTTP/1.1" 200 OK
